# Literature report experiments notebook

This notebook is the output of our initial study during the literature report part of the project.
It implements numerical solution to a 1D wave equation with cubic damping using Harmonic Balance method
and Finite difference method for comparison of the two.

In [1]:
using ModelingToolkit;
using MethodOfLines;
using DifferentialEquations;
using DomainSets;
using FFTW;
using CairoMakie;
using LinearAlgebra;
using BoundaryValueDiffEq;
using SparseArrays;
using NonlinearSolve;
using GLMakie;
include("helper_functions.jl");

In [2]:
xleft::Float64 = 0.0;
xright::Float64 = 1.0;
t0::Float64 = 0.0;
T::Float64 = 150.0;
dt = 0.01;
Nx = 100;
Nt = T / dt;
order = 2;
step = (xright - xleft)/Nx;
grid = collect(xleft:step:xright);

In [3]:
#Definition of constants (placeholders):
gamma::Float64 = 0.0;
gamma_3::Float64 = 2.0;
c::Float64 = 3;
omega = 40;
A_forcing::Float64 = 1000;
lambda_forcing::Float64 = -40;

In [4]:
@parameters x,t
Dx = Differential(x);
Dt = Differential(t);
Dtt = Differential(t)^2;
Dxx = Differential(x)^2;

## Harmonic Balance
To further delve into Julia ecosystem the first part is symbollic calculation, which calculates the coeffitients
for the $\sin$ and $\cos$ functions in first order Harmonic Balance approximation. Then, the equations are 
discretized using a method found in `helper_functions.jl` file, and solved.

In [5]:
@variables A(..), B(..);
r1 = @rule cos(~x)^3 => 0.75 * cos(~x) + 0.25 * cos(3 * ~x);
r2 = @rule sin(~x)^3 => 0.75 * sin(~x) - 0.25 * sin(3 * ~x);
r3 = @rule cos(~x)^2 => 1 - sin(~x)^2;
r4 = @rule sin(~x)^2 => 1 - cos(~x)^2;

In [6]:
@time begin
    u = A(x) * sin(omega*t) + B(x) * cos(omega*t)
    
    y = Dt(Dt(u)) - c*c*Dx(Dx(u)) + gamma*Dt(u) + gamma_3*Dt(u)*Dt(u)*Dt(u)
    y_exp = expand_derivatives(y)
    
    y_exp = simplify(expand(y_exp), RuleSet([r1, r2, r3, r4]))
    y_exp = expand(y_exp)
    y_exp = simplify(y_exp, RuleSet([r1, r2, r3, r4]))
    
    sin_coeff = -Symbolics.coeff(y_exp, sin(omega*t));
    cos_coeff = -Symbolics.coeff(y_exp, cos(omega*t));
    
    println("Trial substitution:");
    println(transform_sym(sin_coeff));
    
    sin_eq_str = transform_sym(sin_coeff)
    cos_eq_str = transform_sym(cos_coeff)
    
    nonlinear_residual = create_residual_function_1D(Nx, sin_eq_str, cos_eq_str)
    initial_guess = ones(2*(Nx+1))
    
    prob = NonlinearProblem(nonlinear_residual, initial_guess, step)
    sol = NonlinearSolve.solve(prob, NewtonRaphson(), reltol = 1e-5, abstol = 1e-5);
    
    println("Solved")
end

Trial substitution:
1600.0*A_array[i_local] + 9.0*(A_array[i_local+1] - 2*A_array[i_local] + A_array[i_local-1]) / dx^2 + 96000.0*((A_array[i_local])^2)*B_array[i_local] + 96000.0*((B_array[i_local])^3)
Solved
  8.318055 seconds (23.11 M allocations: 1.167 GiB, 2.61% gc time, 98.28% compilation time: 13% of which was recompilation)


In [7]:
myA  = sol.u[1:Nx+1];
myB  = sol.u[Nx+2:end];

In [8]:
myA

101-element Vector{Float64}:
  1.0749307269579403e-16
 -0.03724533808630602
 -0.06298567688779776
 -0.07749540157630355
 -0.08186029967678551
 -0.07781117060029112
 -0.06742115797756854
 -0.0527900677416504
 -0.03579961272425125
 -0.017972956830905953
 -0.00043190516864436915
  0.016075693205635164
  0.031109856913968892
  ⋮
 -0.03159230970181645
 -0.03156811084898078
 -0.030781185606229545
 -0.029270486956671005
 -0.027093739660105892
 -0.024323075269876558
 -0.021040328148977373
 -0.017332565234908063
 -0.013288362804179894
 -0.008995206730159442
 -0.0045381958159488485
 -7.655521287414221e-18

In [ ]:
frames = 500

# Observable for u(x,t)
u = Observable(myA .* sin(0.0) .+ myB .* cos(0.0))

# Create figure and axis
fig = Figure(size = (800, 500))
ax = Axis(fig[1, 1],
    title = "t = 0.0",
    xlabel = "x",
    ylabel = "u(x, t)",
    limits = (nothing, nothing, -0.5, 0.5)
)

# Plot linked to observable
lines!(ax, grid, u)

record(fig, "figures/1D_HB_wave.gif", 1:frames; framerate = 60) do i
    t = i * dt
    u[] = myA .* sin(omega * t) .+ myB .* cos(omega * t)
    ax.title = "t = $(round(t, digits = 2))"
end

display("image/gif", read("figures/1D_HB_wave.gif"))

## 

As will be apparent from the next part of the notebook, this solution is very simillar to the Backwards Euler solution visually, but to further examine that those solutions match,
the next graph shows a spatial Fourier transform of the wave (as it is a standing one), to show the underlying frequencies.

In [ ]:
k = 1339

fs = 1/step

new_u = myA * sin(omega * (k * dt)) .+ myB * cos(omega * (k * dt))

F = fftshift(fft(new_u))
freqs =  fftshift(fftfreq(length(new_u), fs))
# Plotting with Makie
f = Figure()
ax1 = Axis(f[1, 1],
    xlabel = "Distance (m)",
    ylabel = "Displacement (m)",
)
ax2 = Axis(f[2, 1], xlabel = "Frequency (Hz)",
    ylabel = "Amplitude (m)",
)
# positions of the dashed lines
peak_freq = freqs[argmax(abs.(F))]
vline_positions = [-peak_freq, peak_freq]
# plot the lines
lines!(ax1, grid, new_u)
lines!(ax2, freqs, abs.(F) / length(new_u))
vlines!(ax2, vline_positions; linestyle = :dash, linewidth = 2, color = :orange)
# set x-limits
xlims!(ax2, -10, 10)
tick_positions = [-10, -5, -peak_freq, 0, peak_freq, 5, 10]
tick_labels = ["-10", "-5", string(round(-peak_freq, digits=2)), "0", string(round(peak_freq, digits=2)), "5", "10"]
ax2.xticks = (tick_positions, tick_labels)

save("figures/space_fourier_HB_linear.png", f)
f

In [ ]:
u_at_point = Float64[]

for i in 0:dt:T - dt
    push!(u_at_point, myA[50] * sin(omega * i) + myB[50] * cos(omega * i))
end

fs_time = 1/dt
N = length(u_at_point)

F = fftshift(fft(u_at_point))
freqs = fftshift(fftfreq(N, fs_time))

# Plotting with Makie
f = Figure()
ax2 = Axis(f[1, 2],
    xlabel = "Frequency (Hz)",
    ylabel = "Amplitude (m)",
)
ax1 =Axis(f[1, 1],
    ylabel = "Amplitude (m)",
    xlabel = "Time (s)",
)
peak_freq = freqs[argmax(abs.(F))]
vline_positions = [-peak_freq, peak_freq]

# Correct normalization: divide by N, not fs_time
lines!(ax1, 0:dt:(T -dt), u_at_point)
lines!(ax2, freqs, abs.(F) ./ N)
xlims!(ax2, -10, 10)

save("figures/time_fourier_HB_linear.png", f)

println("Maximum of ", maximum(abs.(F))./ length(u_at_point), " at frequency of ", abs.(peak_freq))


f

## Finite Difference (Backwards Euler time-integration)

In [ ]:
@variables u(..)

#Define source function (placeholder constant spatial term):
source_f(x, t) = A_forcing * exp(lambda_forcing*(x)^2) * sin(omega*t)

#Define initial condition (placeholder sinusoidal):
#u0(x, t) = exp(-(35*pi)*(x-0.5)^2)
u0(x, t) = 0
v0(x) = 0


#Define equation (with and without cubic damping term):
equation = [Dtt(u(x,t)) + gamma*Dt(u(x,t)) + gamma_3*(Dt(u(x,t)))^3 ~ c^2*Dxx(u(x,t)) + source_f(x, t)]
#equation = [Dtt(u(x,t)) ~ c^2*Dxx(u(x,t))]


#Define domains:
domains = [x ∈ Interval(xleft, xright), t ∈ Interval(t0, T)]

#Define boundary conditions:
bound_conditions = [u(x, 0) ~ u0(x, 0),
                     u(xleft, t) ~ 0,
                     u(xright, t) ~ 0,
                     Dt(u(x, 0)) ~ v0(x)]

In [ ]:
@time begin
    
    #Create PDE system and assign identifier (order matters: eq, bcs, domains, paramers, variables; format parameters and variables as lists, provide arguments whenever they exist):
    @named pde_sys = PDESystem(equation, bound_conditions, domains, [x, t], [u(x,t)]);
    
    #Define a discretization (first argument specifies parameters to discretize, second indicates parameters to keep continuous, third provides the approximation order):
    discretization = MOLFiniteDifference([x=>grid], t, approx_order=order);
    
    #Proceed with discretization:
    @time eq_discretized = discretize(pde_sys,discretization);
    
    #Solve the resulting system (specified solver dictates whether or not time marching is used, as well as the method: Euler(), ImplicitEuler(), or Trapezoidal(), all with given time steps):
    #Warning: if the time step is not explicitly given, solver may default to actual method of lines, instead of time marching\
    dt = 1e-2;
    solution= solve(eq_discretized, ImplicitEuler(), dt=dt, saveat=0.01);
end

In [ ]:
#Plot results:
discrete_t = solution.t

sol_u = solution[u(x, t)]

Nx = size(sol_u, 1)
Nt = length(solution.t)

# Observable for u(x,t)
live = Observable(sol_u[:, 1])

# Create figure and axis
fig = Figure(size = (800, 500))
ax = Axis(fig[1, 1],
    title = "t = 0.0",
    xlabel = "x",
    ylabel = "u(x, t)",
    limits = (nothing, nothing, -0.5, 0.5)
)

# Plot linked to observable
lines!(ax, grid, live)

frames = Nt

record(fig, "figures/1D_FD_wave.gif", 1:500; framerate = 60) do i
    live[] = sol_u[:,i]
    ax.title = "t = $(round(discrete_t[i], digits = 2))"
end

display("image/gif", read("figures/1D_FD_wave.gif"))

In [ ]:
# Fourier Transform analysis

k = 1000
fs = 1/step

u_at_k = sol_u[1:end, k]

F = fftshift(fft(u_at_k))
freqs =  fftshift(fftfreq(length(u_at_k), fs))

# Plotting with Makie
f = Figure()
ax1 = Axis(f[1, 1],
    xlabel = "Distance (m)",
    ylabel = "Displacement (m)",
)

ax2 = Axis(f[2, 1], xlabel = "Frequency (Hz)",
    ylabel = "Amplitude (m)",
)

peak_freq = freqs[argmax(abs.(F))]
vline_positions = [-peak_freq, peak_freq]

lines!(ax1, grid, u_at_k)
lines!(ax2, freqs, abs.(F) / maximum(abs.(F)))
xlims!(ax2, -10, 10)

vlines!(ax2, vline_positions; linestyle = :dash, linewidth = 2, color = :orange)
# set x-limits
xlims!(ax2, -10, 10)
tick_positions = [-10, -5, -peak_freq, 0, peak_freq, 5, 10]
tick_labels = ["-10", "-5", string(round(-peak_freq, digits=2)), "0", string(round(peak_freq, digits=2)), "5", "10"]
ax2.xticks = (tick_positions, tick_labels)

save("figures/space_fourier_time_march_linear.png", f)

f

In [ ]:
delta = 10000
u_at_point = sol_u[50, delta:end]

fs_time = fs = 1/dt

F = fftshift(fft(u_at_point))
freqs =  fftshift(fftfreq(length(u_at_point), fs_time))

# Plotting with Makie
f = Figure()
ax2 = Axis(f[1, 2], xlabel = "Frequency (Hz)",
    ylabel = "Amplitude (m)",
)
ax1 = Axis(f[1, 1], xlabel = "Time (s)",
    ylabel = "Amplitude (m)",
)

peak_freq = freqs[argmax(abs.(F))]
vline_positions = [-peak_freq, peak_freq]
# vlines!(ax2, vline_positions; linestyle = :dash, linewidth = 2, color = :orange)

lines!(ax1, 0:dt:(T - delta*dt + dt), u_at_point)
lines!(ax2, freqs, abs.(F) ./ length(u_at_point))
xlims!(ax2, -10, 10)

tick_positions = [0, 20, 40]
tick_labels = ["100", "120", "140"]
ax1.xticks = (tick_positions, tick_labels)


save("figures/time_fourier_time_march_linear.png", f)

f
println("Maximum of ", maximum(abs.(F))./ length(u_at_point), " at frequency of ", abs.(peak_freq))
